# Binoxxo Puzzle

Place X or O in the empty cells so that there are no more than two consecutive X's or O's in a row or a column.
The number of X's is the same as the number of O's in each row and column, and all rows and columns are unique.

Find more Binoxxo puzzles [here](https://www.binoxxo.ch/Binoxxo-Raetselbuch/)

Imports

In [1]:
from ortools.constraint_solver import pywrapcp
from itertools import product
import numpy as np

Pretty-print square board

In [2]:
def pretty_print(board):
    mapper = {0: 'O', 1: 'X'}
    for i in range(len(board)):
        for j in range(len(board)):
            print("[{}] ".format(mapper[board[i][j].Value()]), end='')
        print("\n")
    print("\n\n")
    
def pretty_print_game(board):
    mapper = {"": '?', "1": 'X', "0": 'O'}
    for i in range(len(board)):
        for j in range(len(board)):
            print("[{}] ".format(mapper[board[i][j]]), end='')
        print("\n")
    print("\n\n")

Binoxxo puzzle from lecture

In [3]:
#Solutions: 1
#Runtime:   239ms
#Failures:  219
#Branches:  438
binoxxo1 = [
    ["", "1", "", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["", "1", "1", "", "", "", "", "", "", ""],
    ["", "", "", "", "0", "0", "", "", "", "0"],
    ["1", "", "", "", "", "", "1", "1", "", ""],
    ["", "1", "", "", "1", "", "", "", "", ""],
    ["", "", "", "0", "", "", "1", "", "", ""],
    ["", "0", "", "", "", "", "", "0", "", "0"],
    ["", "", "", "", "0", "", "", "", "", ""],
    ["0", "", "", "", "", "", "", "", "", "0"]
]

And two more examples

In [4]:
#Solutions: 1
#Runtime:   155ms
#Failures:  486
#Branches:  972 
binoxxo2 = [
    ["", "", "0", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["0", "", "", "", "1", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", ""],
    ["0", "", "", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "", "", "", "", "", "1", "1", ""],
    ["1", "", "", "", "", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", "0"],
    ["", "", "", "", "1", "", "0", "", "", ""]
]

#Solutions: 3
#Runtime:   166ms
#Failures:  0
#Branches:  4 
binoxxo3 = [
    ["1", "", "", "", "", "", "0", "0", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "0", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "1", "", "", "", "0"],
    ["1", "", "1", "", "", "1", "", "", "1", ""],
    ["", "", "1", "", "1", "", "", "", "", ""],
    ["", "", "", "", "1", "1", "", "", "1", ""],
    ["", "", "", "", "", "", "", "", "", "0"],
    ["", "1", "", "1", "", "", "", "", "", ""],
    ["", "1", "", "", "1", "", "", "1", "1", ""]
]

Pick one of the examples

In [5]:
game = binoxxo1

Create constraint solver

In [6]:
solver = pywrapcp.Solver("Binoxxo")

In [7]:
# Create solution_grid
solution_grid = [[solver.IntVar(0, 1) for x in range(len(game))] for y in range(len(game[0]))]

length = len(game)
sum_result = int(len(game)/2)

#set existing values...
for x in range(length):
    for y in range(length):
        if game[x][y] not in "":
            solver.Add(solution_grid[x][y] == int(game[x][y]))
        

#sum of row or colum has to be the half of the length of a row or column
for i in range(length):
    #rows
    solver.Add(sum_result == solver.Sum([solution_grid[i][y] for y in range(length)]))

    #columns
    solver.Add(sum_result == solver.Sum([solution_grid[x][i] for x in range(length)]))
    

#each row and column should be unique...
for i in range(length-1):
    index_one = i
    index_two = i+1
        
    solver.Add(
        solver.Sum([(solution_grid[index_one][y] * int(2**y)) for y in range(length)]) != 
        solver.Sum([(solution_grid[index_two][y] * int(2**y)) for y in range(length)])
    )

    solver.Add(
        solver.Sum([(solution_grid[x][index_one] * int(2**x)) for x in range(length)]) != 
        solver.Sum([(solution_grid[x][index_two] * int(2**x)) for x in range(length)])
    )


#there should not be two times the same symbol in a row...
for x in range(length):
    for y in range(length-2):
        #rows
        solver.Add(solver.Sum([solution_grid[x][y+n] for n in range(0, 3)]) > 0)
        solver.Add(solver.Sum([solution_grid[x][y+n] for n in range(0, 3)]) < 3)

        #columns
        solver.Add(solver.Sum([solution_grid[y+n][x] for n in range(0, 3)]) > 0)
        solver.Add(solver.Sum([solution_grid[y+n][x] for n in range(0, 3)]) < 3)


Configure solver

In [8]:
# Replace this by a list of all decision variables in your model
all_vars = list(np.concatenate(solution_grid))

db = solver.Phase(all_vars, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [9]:
solver.NewSearch(db)
while solver.NextSolution():
    print("Game:")
    pretty_print_game(game)
    print("Solution:")
    pretty_print(solution_grid)

Game:
[?] [X] [?] [?] [?] [?] [?] [?] [?] [?] 

[?] [?] [?] [O] [?] [?] [?] [?] [?] [?] 

[?] [X] [X] [?] [?] [?] [?] [?] [?] [?] 

[?] [?] [?] [?] [O] [O] [?] [?] [?] [O] 

[X] [?] [?] [?] [?] [?] [X] [X] [?] [?] 

[?] [X] [?] [?] [X] [?] [?] [?] [?] [?] 

[?] [?] [?] [O] [?] [?] [X] [?] [?] [?] 

[?] [O] [?] [?] [?] [?] [?] [O] [?] [O] 

[?] [?] [?] [?] [O] [?] [?] [?] [?] [?] 

[O] [?] [?] [?] [?] [?] [?] [?] [?] [O] 




Solution:
[X] [X] [O] [X] [O] [O] [X] [O] [O] [X] 

[X] [O] [X] [O] [X] [O] [O] [X] [O] [X] 

[O] [X] [X] [O] [X] [X] [O] [O] [X] [O] 

[X] [X] [O] [X] [O] [O] [X] [O] [X] [O] 

[X] [O] [O] [X] [O] [O] [X] [X] [O] [X] 

[O] [X] [X] [O] [X] [X] [O] [X] [O] [O] 

[O] [O] [X] [O] [O] [X] [X] [O] [X] [X] 

[X] [O] [O] [X] [X] [O] [X] [O] [X] [O] 

[O] [X] [O] [X] [O] [X] [O] [X] [O] [X] 

[O] [O] [X] [O] [X] [X] [O] [X] [X] [O] 






Cleanup

In [10]:
solver.EndSearch()

Print solver information

In [11]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   161ms
Failures:  219
Branches:  438 
